# Deliverable 3. Create a Travel Itinerary Map.
----
1. Import the vacation CSV file. 
2. From the map above with pick 4 cities that are in close proximity (on the same continent) that the customer would travel to and create a directions layer map.
3. For the "travel_mode" use either "DRIVING", "BICYCLING", or "WALKING".
4. Take a screenshot of the route and save it. 
5. Then create a marker layer for the four cities. Each city will should have a pop-up marker that contains
    - The hotel name
    - The city
    - The country
    - The current weather description and the maximum temperature
---

## Make sure the initial dependencies and the Google API key are imported.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import gmaps.datasets

# Import API key
from config import g_key

In [2]:
# Configure gmaps
gmaps.configure(api_key=g_key)

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [3]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")

# Display sample data
vacation_df.head()

,Unnamed: 0,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,0,Luzhou,CN,80.91,broken clouds,28.8724,105.4405,Shijichun Hotel
1,2,Mae Sai,TH,73.71,overcast clouds,20.4335,99.8762,Piyaporn Hill Paradise Hotel
2,4,Shingu,JP,78.03,scattered clouds,33.7333,135.9833,Shingu UI Hotel
3,6,Kota Bahru,MY,78.75,moderate rain,6.1333,102.2386,Grand Riverview Hotel
4,7,Grimshaw,CA,65.82,few clouds,56.1834,-117.6027,Coast Grimshaw Hotel & Suites


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

## In this step, you will set-up the pop-up markers. Review the code to create a marker layer map of the vacation search results. This code is the same as in Deliverable 2.

In [4]:
# Review the formatting template
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and {Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [5]:
# Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

## From the vacation search map, *choose four cities* that a customer might want to visit. They should be close together and in the same country.

In [6]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# The starting and ending city should be the same city.
vacation_start = vacation_df.loc[(vacation_df["City"] == "Huarmey") & (vacation_df["Country"] == "PE")]
vacation_end = vacation_start
vacation_stop1 = vacation_df.loc[(vacation_df["City"] == "Hualmay") & (vacation_df["Country"] == "PE")]
vacation_stop2 = vacation_df.loc[(vacation_df["City"] == "Lima") & (vacation_df["Country"] == "PE")]
vacation_stop3 = vacation_df.loc[(vacation_df["City"] == "Pisco") & (vacation_df["Country"] == "PE")]

## Use the `to_numpy()` function and list indexing to write code to retrieve the latitude-longitude pairs as tuples from each city DataFrame.

In [7]:
# Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.

def get_lat_lng(vacation_spot):
    return (vacation_spot["Lat"].to_numpy()[0], vacation_spot["Lng"].to_numpy()[0])

In [8]:
start = get_lat_lng(vacation_start)
end = get_lat_lng(vacation_end)
stop1 = get_lat_lng(vacation_stop1)
stop2 = get_lat_lng(vacation_stop2)
stop3 = get_lat_lng(vacation_stop3)

## Use the [gmaps documentation](https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html#directions-layer) to create a directions layer map using the variables from the previous step. Where the starting and ending city are the same city, the `waypoints` are the three other cities, and the `travel_mode` is either `"DRIVING"`, `"BICYCLING"`, or `"WALKING"`.

In [9]:
# Define a new figure object
fig = gmaps.figure()

# Create a direction layer map using the start and end latitude-longitude pairs, and stop1, stop2, and stop3 as the waypoints.
# The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
vacation_itinerary = gmaps.directions_layer(start, end, waypoints=[stop1, stop2, stop3], travel_mode="DRIVING")

# Add the layer to the map
fig.add_layer(vacation_itinerary)

# Display the map
fig

Figure(layout=FigureLayout(height='420px'))

## Use the provided `concat()` function code snippet to combine the four separate city DataFrames into one DataFrame.

In [10]:
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat(
    [
        vacation_start,
        vacation_stop1,
        vacation_stop2,
        vacation_stop3
    ],
    ignore_index = True
)

# Display sample data
itinerary_df

,Unnamed: 0,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,443,Huarmey,PE,62.60,scattered clouds,-10.0681,-78.1522,Hotel Miramar
1,520,Hualmay,PE,62.06,broken clouds,-11.0964,-77.6139,Hotel Villa Kitzia Huacho
2,581,Lima,PE,61.90,overcast clouds,-12.0432,-77.0282,Hotel Kamana
3,197,Pisco,PE,60.85,clear sky,-13.7000,-76.2167,Hotel San Isidro Oficial


## Refactor the code from Step 6 to create a new marker layer map of the cities on the travel route.

In [11]:
# Review the formatting template
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and {Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list
itinerary_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [12]:
# Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=itinerary_info)

# Define a new figure object
fig = gmaps.figure()

# Add the layer to the map
fig.add_layer(marker_layer)

# Display the map
fig

Figure(layout=FigureLayout(height='420px'))